In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
from proq_gen.generators.chroma import get_db_store
from proq_gen.generators.ideation import get_ideation_chain
from proq_gen.generators.generator import get_generator_chain


with open("data/python_kb.json") as f:
    python_kb = json.load(f)

ideation_chain = get_ideation_chain(
    lang="python",
    concept_groups=[
        (
            python_kb["data_processing"]["concepts"],
            6,
        ),  # randomly picks 6 concepts from all concepts
        (
            python_kb["data_formats"],
            5,
        ),  # randomly picks 5 data formats from all data formats
    ],
    **python_kb["data_processing"]["ideation"]
)

db_store = get_db_store("python-questions")

generator_chain = get_generator_chain(ideation_chain, db_store)

/home/necto/micromamba/envs/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
generator_chain.invoke({"n_problems":10})

{'new_problems': ['6718afce-82a3-4252-ab5b-541c52e229c7',
  'ddef8e08-5a38-4cb4-a16c-a91ee3e6c4ef',
  '36e07cae-abc4-4033-a667-c1df8eda7f52',
  '2104c367-b9e0-4693-80b9-e02e8713ad9e',
  '05ec70dd-8b8e-44b5-aee6-9c5cb9031171',
  '9ecd368f-d110-4636-aea6-389421ed56e7',
  '15ce9a70-2ceb-469b-ac66-e424e9dc9c73',
  '53a18c00-c959-42fb-b74c-e3aa3dee6f4a'],
 'duplicate_problems': [{'problem': {'statement': 'Given a list of integers, find the sum of the squares of the numbers greater than the average of the list.',
    'solution': 'def sum_squares_greater_than_avg(nums:list[int])->int:\n    avg = sum(nums)/len(nums)\n    return sum(map(lambda x: x**2, filter(lambda x: x > avg, nums)))',
    'tags': ['numeric', 'map', 'aggregation', 'filtering'],
    'data_formats': ['int', 'list']},
   'statement1': 'Given a list of integers, find the sum of the squares of the numbers greater than the average of the list.',
   'statement2': 'Given a list of integers, find the sum of the squares of the numbers 